In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.io as sio
import scipy.sparse as scisparse
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler
from multiprocessing import Pool

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

NT, NT0    = 60, 30
NX, NY, NZ = 160, 160, 5
DX, DY, DZ = 250, 250, 40
PARENT_DIR = '/mnt/e/Placement-Control-Optim-CO2'
LOCAL_DIR  = 'SlopingAquifer'
sec2year   = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76
co2_rho    = 686.5266
mega       = 1e6
mt2m3      = sec2year*co2_rho/mega/1e3

***

In [ ]:
structuralTraps = sio.loadmat('structuralTraps.mat', simplify_cells=True)['ts']
grid = sio.loadmat('G.mat', simplify_cells=True)['G']
gridVE = sio.loadmat('Gt.mat', simplify_cells=True)['Gt']
tops2d = -gridVE['cells']['z'].reshape(NX,NY)

fig, axs = plt.subplots(1, 2, figsize=(10,5))
im1 = axs[0].imshow(structuralTraps['z_spill_loc'].reshape(NX,NY), cmap='nipy_spectral')
im2 = axs[1].imshow(tops2d, cmap='gist_earth', interpolation='none')
[plt.colorbar(i, pad=0.04, fraction=0.046) for i in [im1, im2]]
[a.set(title=['Spill Points', 'Tops (Elevation)'][i]) for i,a in enumerate(axs)]
plt.tight_layout()
plt.show()

In [ ]:
features = np.zeros((10, 5, NX, NY, NT0))
targets1 = np.zeros((10, 2, NX, NY, NT0))
targets2 = np.zeros((10, 1, NX, NY, NT0))

for i in tqdm(range(10), desc='Processing Data', unit='realization'):
    tops = np.repeat(np.expand_dims(tops2d, -1), NT0, axis=-1)

    rock = sio.loadmat('rock/VE2d/mat/rock2d_{}.mat'.format(i), simplify_cells=True)['var']
    perm = np.repeat(np.expand_dims(rock['perm'].reshape(NX,NY), -1), NT0, axis=-1)
    poro = np.repeat(np.expand_dims(rock['poro'].reshape(NX,NY), -1), NT0, axis=-1)

    ww = sio.loadmat('well_locs/well_locs_{}.mat'.format(i), simplify_cells=True)['var'] - 1
    cc = sio.loadmat('controls/controls_{}.mat'.format(i), simplify_cells=True)['var'] * sec2year * co2_rho / 1e3 / mega

    if len(cc.shape) == 1:
        cc = cc.reshape(1, -1)
    if len(ww.shape) == 1:
        ww = ww.reshape(1, -1)

    well_locs = np.zeros((NX,NY))
    well_locs[ww[:,1], ww[:,0]] = 1
    well_locs = np.repeat(np.expand_dims(well_locs, -1), NT0, axis=-1)

    controls = np.zeros((NX,NY,NT0))
    controls[ww[:,1], ww[:,0], :] = cc

    pressure = np.zeros((NX,NY,NT))
    saturation = np.zeros((NX,NY,NT))
    dd = sio.loadmat('states/states_{}.mat'.format(i), simplify_cells=True)['var']
    for j in range(NT):
        pressure[..., j] = dd[j]['pressure'].reshape(NX,NY)
        saturation[..., j] = dd[j]['s'].reshape(NX,NY)
    
    features[i] = np.stack([poro, perm, tops, well_locs, controls], axis=0)
    targets = np.stack([pressure, saturation], axis=0)
    targets1[i] = targets[..., :NT0]
    targets2[i] = targets[-1,..., NT0:]

print('features: {} | targets1: {} | targets2: {}'.format(features.shape, targets1.shape, targets2.shape))

In [ ]:
fig, axs = plt.subplots(5, 10, figsize=(15,5), sharex=True, sharey=True)
for i in range(5):
    for j in range(10):
        ax = axs[i,j]
        ax.imshow(features[j,i,:,:,-1], cmap='turbo')
        ax.axis('off')
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(3, 10, figsize=(15,4), sharex=True, sharey=True)
for j in range(10):
    for i in range(2):
        ax = axs[i,j]
        ax.imshow(targets1[j,i,:,:,-1], cmap='turbo')
    ax2 = axs[2,j]
    ax2.imshow(targets2[j,0,:,:,-1], cmap='turbo')        
plt.tight_layout()
plt.show()

In [2]:
from utils import NeuralOperator

In [3]:
fno1 = NeuralOperator(in_channels=5, out_channels=2, modes1=16, modes2=16, modes3=16, width=64, kernel_size=3)
fno2 = NeuralOperator(in_channels=2, out_channels=1, modes1=16, modes2=16, modes3=16, width=64)

In [4]:
temp = torch.rand((1,NX,NY,NT0,5))

temp_out = fno1(temp)
print(temp_out.shape)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 15 but got size 16 for tensor number 1 in the list.

In [ ]:
totVol = []
vol = []
freeVol = []
trappedVol = []
leakedVol = []
timesteps = []

for i in tqdm(range(1, 60)):
    d = dd[i]
    totVol.append(d['totVol']/mt2m3)
    vol.append(d['vol']/mt2m3)
    freeVol.append(d['freeVol']/mt2m3)
    trappedVol.append(d['trappedVol']/mt2m3)
    leakedVol.append(d['leakedVol']/mt2m3)
    timesteps.append(d['t'])

totVol = np.array(totVol)
vol = np.array(vol)
freeVol = np.array(freeVol)
trappedVol = np.array(trappedVol)
leakedVol = np.array(leakedVol)
timesteps = np.array(timesteps)

In [ ]:
fig = plt.figure(figsize=(10,7))
ax  = fig.add_subplot(111)
im = ax.imshow(cc, cmap='turbo', interpolation='none')
ax.set(yticks=range(cc.shape[0]), yticklabels=['c{}'.format(i) for i in range(1, cc.shape[0]+1)],
       xticks=range(cc.shape[1]), xticklabels=range(1, cc.shape[1]+1),
       xlabel='Time step', ylabel='Control')
plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.084, fraction=0.046)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
for i in range(30):
    plt.subplot(2, 15, i+1)
    s = dd[i*2]['s'].reshape(NX,NY) 
    p = dd[i*2]['pressure'].reshape(NX,NY) / psi2pascal
    plt.imshow(s, cmap='jet')
    plt.scatter(ww[:,0], ww[:,1], c='w', edgecolor='k', s=10, marker='v')
    plt.axis('off')
    plt.title('t={}'.format(dd[i*2+1]['t']))
plt.tight_layout()
plt.show()

In [ ]:
sat = np.zeros((60, NX, NY))

for i in range(60):
    sat[i] = dd[i]['s'].reshape(NX,NY)

In [ ]:
nw = []
for i in tqdm(range(1272)):
    a = sio.loadmat('well_locs/well_locs_{}.mat'.format(i), simplify_cells=True)['var']
    if len(a.shape) == 1:
        a = a.reshape(1, -1)
    nw.append(a.shape[0])

In [ ]:
pv = np.ones((160,160))*(DX*DY*DZ)

In [ ]:
pv @ 

In [ ]:
np.sum(sat[0] @ ) / co2_rho / sec2year * 1e3 * mega

In [ ]:
totVol[0]

In [ ]:
np.max(sat[0])

In [ ]:
np.sum( sat[0]) / (DX*NX*DY*NY*DZ*NZ) 

In [ ]:
(totVol[1] - totVol[0]) / (DX*DY*DZ)

In [ ]:
np.sum(totVol) * co2_rho * sec2year / 1e3 / mega

In [ ]:
print('Trapped:', np.sum(trappedVol))
print('Leaked:', np.sum(leakedVol))
print('Total:', np.sum(totVol))

In [ ]:
trapped_labels = ['residual (traps)', 'residual', 'free (residual)', 'moveable', 'free plume', 'leaked']

plt.figure(figsize=(18,4))

plt.plot(timesteps, totVol, label='Total Volume', c='tab:green')
plt.plot(timesteps, np.sum(vol, 1), label='Sum of Vols', c='tab:blue')

plt.plot(timesteps, np.sum(vol,1)+leakedVol, label='Sum of Vols + Leaked', ls='', marker='.', c='g')
plt.plot(timesteps, np.sum([freeVol, trappedVol], 0), label='Free + Trapped', ls='', marker='.', c='b')

plt.plot(timesteps, leakedVol, label='Leaked Volume', c='r', marker='x')
plt.plot(timesteps, trappedVol, label='Trapped Volume', c='tab:orange', marker='s')
plt.plot(timesteps, freeVol, label='Free Volume', c='magenta', marker='.')

plt.plot(timesteps, freeVol+trappedVol+leakedVol, c='k', label='Free + Trapped + Leaked')

# vols = np.hstack([vol, leakedVol.reshape(-1,1)])
# for k in range(6):
#     plt.plot(timesteps, vols[:,k], label=trapped_labels[k], ls='-.', c='C{}'.format(k+4))

plt.legend(facecolor='lightgrey', edgecolor='k', loc='upper left')
plt.xscale('log')
plt.grid(True, which='both')

plt.show()

In [ ]:
from utils import LpLoss, NeuralOperator